In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'].dt.normalize() - pd.Timedelta(days=5)

In [10]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

# Checks

In [11]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [12]:
check2 = costo_venta[costo_venta['product_price'].isna()][['product_id', 'product_name_x']]
print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 21 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10282,28586,Anticipo
13416,28638,Servicios de Facturación
18005,4,Anticipo (PdV)


In [13]:
#Variaciones del precio en productos varios proveedor vs un sólo proveedor

# Pruebas

In [14]:
prod_varios_prov = (compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']]
                    .sort_values('partner_id').groupby('product_id_pp')
                    .agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
                    )

prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']

prod_varios_prov_ids = prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']

ventas_prod_varios_prov = ventas_año[ventas_año['product_id'].isin(prod_varios_prov_ids)]
compras_prod_varios_prov = compras[compras['product_id_pp'].isin(prod_varios_prov_ids)]

ventas_año_diff = ventas_año[~ventas_año['product_id'].isin(prod_varios_prov_ids)]

In [15]:
compras_prod_varios_prov['product_qty'] = compras_prod_varios_prov['product_qty'].astype('Float64')
compras_prod_varios_prov['vendedora'] = compras_prod_varios_prov['vendedora'].convert_dtypes()

C:\Users\Lic.Costich\AppData\Local\Temp\ipykernel_10972\107290796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compras_prod_varios_prov['product_qty'] = compras_prod_varios_prov['product_qty'].astype('Float64')
C:\Users\Lic.Costich\AppData\Local\Temp\ipykernel_10972\107290796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compras_prod_varios_prov['vendedora'] = compras_prod_varios_prov['vendedora'].convert_dtypes()


In [16]:
# Checar si es mejor primero (ronda 1 de concatenación), sacar de las compras los 212 'prod_varios_prov_ids', para que se quede el resto del DF en esa columna como NaN y en una
# segunda vuelta calificar por niveles el resto de los renglones NaN

v = pd.concat([ventas_año_diff, ventas_prod_varios_prov]).sort_values(by='invoice_date')

### 2da vuelta al costo de ventas

In [17]:
compras_prod_varios_prov_odoo = compras_prod_varios_prov[~compras_prod_varios_prov['line_id'].isna()]

In [18]:
# ventas_costo_venta_prod_varios_prov = (
#     pd.merge_asof(
#         left = ventas_prod_varios_prov.sort_values('invoice_date'),
#         right = compras_prod_varios_prov.sort_values('order_date'), 
        
#         left_by = ['product_id', 'salesperson_name', 'quantity'], 
#         right_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        
#         left_on = 'invoice_date', 
#         right_on = 'order_date', 

#         tolerance = pd.Timedelta(days=5),
        
#         direction = 'nearest')
# )

In [19]:
compras_costo_venta_prod_varios_prov = (
    pd.merge_asof(
        left = compras_prod_varios_prov_odoo.sort_values('order_date'), 
        right = ventas_prod_varios_prov.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (compras_costo_venta_prod_varios_prov['fact_line_id'].duplicated()), 'fact_line_id']
ids_order_line_con_ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[compras_costo_venta_prod_varios_prov['fact_line_id'].isin(ids_fact_line_repetidas), 'line_id']
costo_ventas_2da_vuelta_merge1 = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (~compras_costo_venta_prod_varios_prov['line_id'].isin(ids_order_line_con_ids_fact_line_repetidas)), ['fact_line_id', 'line_id']]

ventas_prod_varios_prov_after_merge1 = ventas_prod_varios_prov.loc[~ventas_prod_varios_prov['fact_line_id'].isin(costo_ventas_2da_vuelta_merge1['fact_line_id'])]
compras_prod_varios_prov_odoo_after_merge1 = compras_prod_varios_prov_odoo.loc[~compras_prod_varios_prov_odoo['line_id'].isin(costo_ventas_2da_vuelta_merge1['line_id'])]

In [20]:
ids_order_line_same_match = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (compras_costo_venta_prod_varios_prov['line_id'].isin(ids_order_line_con_ids_fact_line_repetidas)), 'line_id']
ids_order_line_same_match

71        396
81        422
404      3425
405      3426
426      3539
442      3803
467      3861
483      4147
521      4931
548      5554
570      5651
577      5729
702      7993
722      8606
773      9343
785      9814
787      9901
802     10086
825     10196
833     10212
919     11468
938     11529
947     11569
1034    13592
1036    13595
Name: line_id, dtype: Int64

In [21]:
compra

{'id': 1,
 'name': 'P00001',
 'state': 'purchase',
 'partner_id': [5326, 'Madereria El Pino De Los Cabos'],
 'partner_ref': 'SJCFCO 68',
 'date_approve': '2024-01-03 00:05:53',
 'x_fecha_factura': False,
 'user_id': [213, 'Yamilet Blanco Salas'],
 'create_uid': False}

In [22]:
#Sólo para explicación, borrar
ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (compras_costo_venta_prod_varios_prov['fact_line_id'].duplicated()), 'fact_line_id']
ids_fact_line_repetidas

81       18752
426     106881
467     114423
483     123517
548     146483
577     157272
722     214630
787     224875
825     229110
833     238504
938     266342
947     266342
1036    304029
Name: fact_line_id, dtype: Int64

In [23]:
#Sólo para explicación, borrar
ids_order_line_con_ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[compras_costo_venta_prod_varios_prov['fact_line_id'].isin(ids_fact_line_repetidas), 'line_id']
ids_order_line_con_ids_fact_line_repetidas

71        396
81        422
404      3425
405      3426
426      3539
442      3803
467      3861
483      4147
521      4931
548      5554
570      5651
577      5729
702      7993
722      8606
773      9343
785      9814
787      9901
802     10086
825     10196
833     10212
919     11468
938     11529
947     11569
1034    13592
1036    13595
Name: line_id, dtype: Int64

In [24]:
#Sólo para explicación, borrar
cols_show = ['line_id', 'order_date', 'product_qty','vendedora',
             'fact_line_id', 'invoice_date', 'quantity', 'salesperson_name']
compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (compras_costo_venta_prod_varios_prov['line_id'].isin(ids_order_line_con_ids_fact_line_repetidas)), cols_show].sort_values(by='fact_line_id')

,line_id,order_date,product_qty,vendedora,fact_line_id,invoice_date,quantity,salesperson_name
71,396,2024-01-11 23:13:19,6.00,Mayra Angelica Parada Manjarrez,18752,2024-01-11,6.00,Mayra Angelica Parada Manjarrez
81,422,2024-01-12 16:05:14,6.00,Mayra Angelica Parada Manjarrez,18752,2024-01-11,6.00,Mayra Angelica Parada Manjarrez
404,3425,2024-02-20 15:31:50,1.00,Yamilet Blanco Salas,106881,2024-02-21,1.00,Yamilet Blanco Salas
426,3539,2024-02-22 00:58:52,1.00,Yamilet Blanco Salas,106881,2024-02-21,1.00,Yamilet Blanco Salas
405,3426,2024-02-20 15:31:50,5.00,Yamilet Blanco Salas,114423,2024-02-24,5.00,Yamilet Blanco Salas
467,3861,2024-02-26 19:41:19,5.00,Yamilet Blanco Salas,114423,2024-02-24,5.00,Yamilet Blanco Salas
442,3803,2024-02-26 15:36:56,2.00,Irma Carvajal Flores,123517,2024-02-28,2.00,Irma Carvajal Flores
483,4147,2024-02-29 17:35:11,2.00,Irma Carvajal Flores,123517,2024-02-28,2.00,Irma Carvajal Flores
521,4931,2024-03-07 00:33:15,5.00,Brenda Luz Acosta Lopez,146483,2024-03-08,5.00,Brenda Luz Acosta Lopez
548,5554,2024-03-09 00:28:01,5.00,Brenda Luz Acosta Lopez,146483,2024-03-08,5.00,Brenda Luz Acosta Lopez


In [25]:
#Sólo para explicación, borrar
costo_ventas_2da_vuelta_merge1 = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (~compras_costo_venta_prod_varios_prov['line_id'].isin(ids_order_line_con_ids_fact_line_repetidas)), ['fact_line_id', 'line_id']]
costo_ventas_2da_vuelta_merge1

,fact_line_id,line_id
0,2576,3
1,2927,2
2,11002,5
5,5709,63
6,5706,62
...,...,...
1373,413988,19093
1374,413987,19092
1377,414614,19110
1378,409125,19113


In [26]:
#Sólo para explicación, borrar
print(len(compras_prod_varios_prov_odoo))
print(len(ventas_prod_varios_prov))

1391
4064


In [27]:
#Sólo para explicación, borrar
print(len(ventas_prod_varios_prov.loc[~ventas_prod_varios_prov['fact_line_id'].isin(costo_ventas_2da_vuelta_merge1['fact_line_id'])]))
print(len(compras_prod_varios_prov_odoo.loc[~compras_prod_varios_prov_odoo['line_id'].isin(costo_ventas_2da_vuelta_merge1['line_id'])]))

3325
652


In [28]:
# For para dar tratamiento a las líneas de compra que tuvieron tuvieron match con una misma línea de venta
wep_nel = []

for i in ids_order_line_same_match:

    compra = compras_prod_varios_prov_odoo.loc[compras_prod_varios_prov_odoo['line_id'] == i].iloc[0]

    mini_df = ventas_prod_varios_prov_after_merge1.loc[
                (ventas_prod_varios_prov_after_merge1['product_id'] == compra['product_id_pp'])
                & (ventas_prod_varios_prov_after_merge1['salesperson_name'] == compra['vendedora'])
                & (ventas_prod_varios_prov_after_merge1['quantity'] == compra['product_qty'])
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] >= compra['order_date'] - pd.Timedelta(days=5))
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] <= compra['order_date'] + pd.Timedelta(days=5))
            ]

    if len(mini_df) > 1:
        wep_nel.append(i)

wep_nel

[3803, 4931, 5554, 9343, 9814, 9901, 11468, 11529, 11569, 13592]

In [29]:
wep_igual_uno = []
wep_muchas = []
wep_vacias = []

for i in range(len(ventas_prod_varios_prov)):
    
    ventas = ventas_prod_varios_prov.iloc[i]

    mini_df = compras_prod_varios_prov_odoo.loc[
                (compras_prod_varios_prov_odoo['product_id_pp'] == ventas['product_id'])
                & (compras_prod_varios_prov_odoo['vendedora'] == ventas['salesperson_name'])
                & (compras_prod_varios_prov_odoo['product_qty'] == ventas['quantity'])
                & (compras_prod_varios_prov_odoo['order_date'] >= ventas['invoice_date'] - pd.Timedelta(days=5))
                & (compras_prod_varios_prov_odoo['order_date'] <= ventas['invoice_date'] + pd.Timedelta(days=5))
            ]

    if len(mini_df) == 1:
        wep_igual_uno.append(i)

    if len(mini_df) > 1:
        wep_muchas.append(i)
    
    if mini_df.empty:
        wep_vacias.append(i)


In [30]:
print(len(wep_igual_uno), 'wep_igual_uno')
print(len(wep_muchas), 'wep_muchas')
print(len(wep_vacias), 'wep_vacias')

747 wep_igual_uno
25 wep_muchas
3292 wep_vacias


In [31]:
wep = (
    compras_costo_venta_prod_varios_prov.loc[
        (~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) 
        & (compras_costo_venta_prod_varios_prov['fact_line_id'].duplicated()),
        'fact_line_id']
)

compras_costo_venta_prod_varios_prov.loc[compras_costo_venta_prod_varios_prov['fact_line_id'].isin(wep)].to_excel('wep.xlsx')


In [32]:
ids_ventas_encontradas_1 = compras_costo_venta_prod_varios_prov[~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()]['fact_line_id']

ventas_prod_varios_prov.loc[ventas_prod_varios_prov['fact_line_id'].isin(ids_ventas_encontradas_1)]

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,warehouse,fact_line_id,product_id,product_name,prod_codigo,prod_linea,quantity,price_unit,discount,price_subtotal
333,722,F2-VS/2024/00002,2024-01-03,posted,S00160,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,2576,14147,Triplay Pino 3/4 4'*8' 1/Cara *3637*,3637,00MAD,20.00,969.82,0.00,"19,396.40"
379,818,F2-VS/2024/00003,2024-01-03,posted,S00110,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,2927,14272,Tablon Americano 2x12x12 *13508*,13508,00MAD,2.00,"1,551.74",0.00,"3,103.48"
816,1546,F1-VS/2024/00023,2024-01-05,posted,S00084,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A1,5685,13636,Tabla Americana 1x12x10 *13756*,13756,00MAD,5.00,581.25,0.00,"2,906.25"
817,1546,F1-VS/2024/00023,2024-01-05,posted,S00084,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A1,5686,14184,Tablon americano 2*12*10 *90057*,90057,00MAD,5.00,"1,673.20",0.00,"8,366.00"
820,1548,F2-CC/2024/00204,2024-01-05,posted,PdV SJC/0261,PdV,478,out_invoice,<NA>,<NA>,...,A2,5693,13959,Triplay Pino 3/4 4'*8' 0/Caras *5790*,5790,00MAD,7.00,816.66,0.00,"5,716.62"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63573,110230,F2-VS/2024/01539,2024-06-26,posted,S29925,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,413987,13943,Triplay Birch 1/2 4'*8' *15143*,15143,00MAD,2.00,775.86,0.00,"1,551.72"
63574,110230,F2-VS/2024/01539,2024-06-26,posted,S29925,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,413988,14075,Triplay Birch 5/8 4'*8' *40032*,40032,00MAD,3.00,955.39,0.00,"2,866.17"
63575,110230,F2-VS/2024/01539,2024-06-26,posted,S29925,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,413989,13331,Triplay Birch 1/4 4'*8' *16320*,16320,00MAD,2.00,413.81,0.00,827.62
63584,110230,F2-VS/2024/01539,2024-06-26,posted,S29925,Ventas,<NA>,out_invoice,<NA>,<NA>,...,A2,413998,13786,"Tabla Americana 1"" X 12"" X 16' *13538*",13538,00MAD,2.00,833.80,0.00,"1,667.60"


In [33]:
len(ventas_prod_varios_prov)-764

3300

In [34]:
len(compras_costo_venta_prod_varios_prov)-764

627

Después de corregir el costo de la venta... las siguientes líneas ya son de utilidades

In [35]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [36]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]